In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import HNLoader
from langchain_groq import ChatGroq
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/ali/Desktop/QA/RAG/loca-rag/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Create embedding model for LangChain
embedding_model = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cpu"},  # یا "cuda" برای GPU
    encode_kwargs={"normalize_embeddings": True}
)



In [3]:


# Step 1: Load the .txt file
loader = TextLoader("/home/ali/Desktop/QA/RAG/sadegh.txt", encoding="utf-8")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]  # optional, helps preserve meaning
)

docs = text_splitter.split_documents(documents) 

In [4]:
base_url = "https://api.avalai.ir/v1"
api_key = ""

#model_name = "gpt-4o"

# استفاده از مدل gemini-2.0-flash-exp
model_name = "gemini-2.0-flash"
#model_name = 'deepseek-chat'
# استفاده از مدل anthropic.claude-3-5-sonnet-20240620-v1:0
# model_name = "anthropic.claude-3-5-sonnet-20240620-v1:0"

llm = ChatOpenAI(
    base_url=base_url,
    model=model_name,
    api_key=api_key,
)



In [35]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  # Specify the embedding model
    base_url="https://api.avalai.ir/v1",  # Your custom endpoint
    api_key=""  # API key for authentication
)





In [5]:
vector = FAISS.from_documents(docs,embedding_model) 
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 8})


In [6]:
print(retriever.invoke('صادق هدایت توپ مرواری را چه زمانی نوشت؟')[0].page_content) 

در سال ۱۳۲۶ به نوشتن توپ مرواری پرداخت اما این اثر تا پس از مرگش به چاپ نرسید. معروف‌ترین نام مستعار او که توپ مرواری هم تحت آن منتشر شد هادی صداقت است. در ۱۳۲۷ مقالهٔ «پیام کافکا» به‌صورت مقدمه‌ای بر کتاب گروه محکومین نوشتهٔ کافکا و ترجمهٔ حسن قائمیان نوشت. در سال ۱۳۲۹ با همکاری حسن قائمیان داستان «مسخ» کافکا را ترجمه کرد و در مجلهٔ سخن انتشار داد. در ۱۲ آذر همان سال با گرفتن گواهی پزشکی (برای اخذ روادید) و فروختن کتاب‌هایش به فرانسه رفت. در طول اقامت در فرانسه سفری به هامبورگ داشت و نیز سعی کرد به لندن برود که [موفّق] نشد. سرانجام در روز دوشنبه ۱۹ فروردین ۱۳۳۰ در آپارتمان اجاره‌ای‌اش — در خیابان شامپیونه، واقع در پاریس — با گاز خودکشی کرد.[۳۹] او نخستین نویسنده و ادیب ایرانی محسوب می‌شود که خودکشی کرده است. وی چند روز قبل از انتحار بسیاری از داستان‌های چاپ‌نشده‌اش را نابود کرده بود. هدایت را در قبرستان پرلاشز پاریس به خاک سپردند. مراسم خاکسپاری‌اش با حضور عده‌ای قلیل از ایرانیان و فرانسویان صورت گرفت.[۴۰]

طراح سنگ آرامگاهِ صادق هدایت، علی سردارافخمی معمار ایرانی است.[۴۱]


Reranking 


In [7]:

# Initialize the compressor
compressor = FlashrankRerank(top_n=4,model = 'ms-marco-MultiBERT-L-12')

# Create the compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=retriever  # Make sure 'retriever' is defined elsewhere in your code
)







Contectual Comprehension

In [76]:
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever
)
compressed_docs = compression_retriever.invoke(
    'صادق هدایت چه زمانی خودکشی کرد؟'
)


INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"


In [8]:
contextualize_q_system_prompt = """
Given a chat history and the latest user question\
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is.
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system',contextualize_q_system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human',"{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(llm,compression_retriever,contextualize_q_prompt)




In [9]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\
create reposponse with Persian language. 
{context}"""


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


trimmer = trim_messages(strategy="last", max_tokens=1000, token_counter=len)

chain_with_trimming = (
    # Replace the current chat_history with the trimmed chat history
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer)
    | rag_chain
)


store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    chain_with_trimming,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)





In [10]:
result = conversational_rag_chain.invoke(
    {"input": "این سند در مورد چیه؟"},
    config={
        "configurable": {"session_id": "a1"}
    },
)["answer"]
print(result)


INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"


این سند درباره صادق هدایت، زندگی، شخصیت، آثار و نقدهایی بر آثار او است. این سند به برخی از جنبه‌های شخصیتی او مانند طنز قوی و انتقادی، گیاه‌خواری و دیدگاهش نسبت به شرح حال خود نیز اشاره می‌کند. همچنین، به تأثیر عمیق هدایت بر نویسندگان پس از خود اشاره دارد.



In [11]:
result = conversational_rag_chain.invoke(
    {"input": "معروف ترین کتاب صادق هدایت بگو"},
    config={
        "configurable": {"session_id": "a1"}
    },
)["answer"]
print(result)




INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"


معروف‌ترین اثر صادق هدایت، رمان بوف کور است که بسیاری از پژوهشگران آن را درخشان‌ترین اثر ادبیات داستانی معاصر ایران دانسته‌اند.



In [12]:
result = conversational_rag_chain.invoke(
    {"input": "چه زمانی به هند سفر کرد؟"},
    config={
        "configurable": {"session_id": "a1"}
    },
)["answer"]
print(result)

INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.avalai.ir/v1/chat/completions "HTTP/1.1 200 OK"


صادق هدایت در سال ۱۳۱۵ به همراه شین پرتو به هند سفر کرد.

